# [Problem](https://codingcompetitions.withgoogle.com/kickstart/round/00000000008f4332/0000000000943934#problem)
- **Given**
    + `N`: binary string size `P`. Eg:
        + 1100
        + 1010
        + 0000
    + `M`: binary string size `P`. Eg:
        + 1000
- **Find** a binary string `target` and return the min cost such that
    + cost = total difference between ans and each string in `Ns`
    + `target` must not appear in `Ms`

#### Input
```
N M P
Ns
Ms
```

#### Constraints

- $1\leq N\leq 100$
- $1\leq M\leq $min(100,$2^P$−1)
- $1\leq P\leq 100$


#### Example 1

```
// Input
3 1 4

1100
1010
0000

1000

// Output
4

// Explanation
target = 1100, min total cost = 4

diff(1100, 1100) = 0
diff(1010, 1100) = 2
diff(0000, 1100) = 2
```

#### Example 2

```
// Input
2 4 4

1111
1111

1111
0111
1011
1101

// Output
2

// Explanation
target = 1110, min total cost = 2

diff(1111, 1110) = 1
diff(1111, 1110) = 1
```

# Solution
#### Brute force 1

```C++
int N, M, P;
vector<string> Ns;
unordered_set<string> Ms;

// min_heap
multimap<int, string> configs;
void generate(const string &config, int cost) {
    if(config.size() == P) {
        configs.insert( {cost, config} );
        return;
    }

    int j = config.size();

    // search 0
    int complaints = 0;
    for(int i=0; i<N; ++i) {
        if(Ns[i][j] != '0') complaints += 1;
    }
    generate(config + '0', cost + complaints);

    // search 1
    complaints = 0;
    for(int i=0; i<N; ++i) {
        if(Ns[i][j] != '1') complaints += 1;
    }
    generate(config + '1', cost + complaints);
}

int sol() {
    configs.clear();

    // dfs
    generate("", 0);

    // Check ans
    for(auto &[cost, config]: configs) {
        if(Ms.count(config) == NULL) return cost;
    }

    assert(false);
    return 0;
}
```

#### Brute force 2

```C++
int N, M, P;
vector<string> Ns;
unordered_set<string> Ms;

int generate(const multimap<int, string> &configs) {
    int j = configs.begin()->second.size();
    
    // Last step
    if(j == P) {
        for(auto &[cost, config]: configs) {
            if(Ms.count(config) == NULL) return cost;
        }
        assert(false);
        return 0;
    }

     // Generate configs 
    multimap<int, string> new_configs;
    for(auto &[cost, config]: configs) {
        // 0
        int complaints = 0;
        for(int i=0; i<N; ++i) {
            if(Ns[i][j] != '0') complaints += 1;
        }
        new_configs.insert( {cost + complaints, config + '0'} );

        // 1
        complaints = 0;
        for(int i=0; i<N; ++i) {
            if(Ns[i][j] != '1') complaints += 1;
        }
        new_configs.insert( {cost + complaints, config + '1'} );
    }
    return generate(new_configs);
}

int sol() {
    // dfs
    multimap<int, string> configs;
    configs.insert( {0, ""} );
    return generate(configs);
}
```

#### Optimized search
- **Insights**: Start from Bruteforce 2
    + Cache the cost/score of `Ns[i][j]`
    + Since we maximumly remove `M` candidate, make a greedy move, store the best `M+1` candidates -> atleast 1 candidate is not rejected
    

```C++
int N, M, P;
vector<string> Ns;
unordered_set<string> Ms;


unordered_map<int, int> pre_score_0, pre_score_1;
int generate(const multimap<int, string> &configs) {
    int j = configs.begin()->second.size();

    // Last step
    if(j == P) {
        for(auto &[cost, config]: configs) {
            if(Ms.count(config) == NULL) return cost;
        }
        assert(false);
        return 0;
    }

    // Generate configs
    multimap<int, string> new_configs;
    for(auto &[cost, config]: configs) {
        // 0
        new_configs.insert( {cost + pre_score_0[j], config + '0'} );

        // 1
        new_configs.insert( {cost + pre_score_1[j], config + '1'} );
    }

    // Greedy: Choose M+1 best cases
    vector<pair<int, string>> tmp(new_configs.begin(), new_configs.end());
    new_configs = tmp.size() > M+1 ? \
        multimap<int, string>(tmp.begin(), tmp.begin()+M+1): \
        multimap<int, string>(tmp.begin(), tmp.end());

    return generate(new_configs);
}

int sol() {
    // cache complaints
    pre_score_0.clear();
    pre_score_1.clear();
    for(int j=0; j<P; ++j) {
        // 0
        int complaints = 0;
        for(int i=0; i<N; ++i) {
            if(Ns[i][j] != '0') complaints += 1;
        }
        pre_score_0[j] = complaints;

        // 1
        complaints = 0;
        for(int i=0; i<N; ++i) {
            if(Ns[i][j] != '1') complaints += 1;
        }
        pre_score_1[j] = complaints;
    }

    // dfs
    multimap<int, string> configs;
    configs.insert( {0, ""} );
    return generate(configs);
}
```